#### Загрузка библиотек

In [469]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [470]:
import numpy as np

In [471]:
from tqdm import tqdm
tqdm.pandas()

In [472]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error as mape

In [473]:
from datetime import datetime
import statsmodels.api as sm

In [474]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

In [475]:
from catboost import CatBoostRegressor

#### Подготовка датасета для обучения

In [ ]:
response = requests.get('https://auto.ru/cars/all/')
response.encoding = 'utf-8'
result = BeautifulSoup(response.text, 'html.parser')

In [ ]:
def string_to_dict(string): ### функция для трансформации строкового представления словаря в собственно словарь
    result = {}
    string = string[1:-1]
#     print(string)
    while len(string) > 0:
#         print(len(string))
        find_comma = string.find(',')
        find_colon = string.find(':')
        find_brackets = string.find('{')
#         print(find_comma, find_colon)
        if find_brackets < find_comma:
            result[string[:find_colon-1]] = string[find_colon+1:string.find('}')+1]
            thr = string.find('}') + 1
        else:
#             print(string[1:find_colon-1])
            if find_colon+1 == '\"':
                if 'description' not in string[1:find_colon-1]:
                    result[string[1:find_colon-1]] = string[find_colon+1+1:find_comma-1]
                    thr = find_comma
                else:
                    result[string[1:find_colon-1]] = string[find_colon+1+1:string.find(',"numberOfDoors":')]
                    thr = string.find(',"numberOfDoors":')
            else:
                if 'description' not in string[1:find_colon-1]:
                    result[string[1:find_colon-1]] = string[find_colon+1:find_comma]
                    thr = find_comma
                else:
                    result[string[1:find_colon-1]] = string[find_colon+1:string.find(',"numberOfDoors":')]
                    thr = string.find(',"numberOfDoors":')            
#         print(result)
        string = string[thr+1:]
#         print(string)
#         print(len(string))
    return result

In [ ]:
### наполнение учебного датасета
response = requests.get('https://auto.ru/cars/all/')
response.encoding = 'utf-8'
result = BeautifulSoup(response.text, 'html.parser')
columns = [i for i in string_to_dict(str(result.select('script')[0])[len('<script type=\"application/ld+json\">'):-len('</script>')].replace('\',"', '').replace('\"https://','')).keys()]
df = pd.DataFrame(columns = columns)
# print(columns)
for i in tqdm(range(1, 100)):
#     print('Page', i)
    url = 'https://auto.ru/cars/all/?page='+str(i)
    response = requests.get(url)
    response.encoding = 'utf-8'
    result = BeautifulSoup(response.text, 'html.parser')    
    for j in range(len(result.select('script'))):
#         print(f'Page {i}, Ad {j}')
        string = str(result.select('script')[j])[len('<script type=\"application/ld+json\">'):-len('</script>')].replace('\',"', '').replace('\"https://','')
#         print(string)
        if 'numberOfDoors' not in string:
            continue
        else:
            auto_dict = {}
            for k in range(len(columns)):                
                ###
#                 print(k, len(columns))
                if k < len(columns) - 1:
#                     print(columns[k], columns[k+1])
#                     print(string[1+len(columns[k])+2:string.find(str(columns[k+1])+'":')-2])
                    auto_dict[columns[k]] = string[1+len(columns[k])+2:string.find(str(columns[k+1])+'":')-2]
                    string = string[string.find(str(columns[k+1])+'":')-1:]
                else:
                    auto_dict[columns[k]] = string[1+len(columns[k])+2:]
#             print(auto_dict)
#             display(pd.DataFrame.from_dict(auto_dict, orient='index').T)
            df = pd.concat([df, pd.DataFrame.from_dict(auto_dict, orient='index').T])

In [477]:
def micro_string_to_dict(string): ### дополняющая функция для трансформации строковых представлений вложенных словарей
    result = {}
    string = string[1:-1]
#     print(string)
    while len(string) > 1:
#         print(len(string))
        find_comma = string.find(',')
        find_colon = string.find(':')
        if string[string.rfind('","')+3:string.rfind(':')] not in string[1:find_colon-1]:
            result[string[1:find_colon-1]] = string[find_colon+1:find_comma]
            thr = find_comma
        else:
            result[string[1:find_colon-1]] = string[find_colon+1:]
            thr = -2
        if thr == -1:
            thr = -2
#         print('Threshold', thr)
#         print('resulting_dict', result)           
#         print(result)
#         print(thr)
        string = string[thr+1:]
#         print(string)
#         print(len(string))
    return result

In [596]:
df = pd.read_csv('project_dataframe.csv')

In [597]:
df = df.drop(columns=['"offers'])

In [598]:
df = df.rename(columns={'name':'vehicleName'})

In [599]:
df.columns = [i.replace('"', '') for i in df.columns]
for i in df.columns:
    print(i)
    df[i] = df[i].progress_apply(lambda x: micro_string_to_dict(x) if (isinstance(x, int) == False and isinstance(x, dict) == False and x[0] == '{') else x)

@context


100%|██████████████████████████████████████████████████████████████████████████| 3532/3532 [00:00<00:00, 585521.59it/s]


@type


100%|██████████████████████████████████████████████████████████████████████████| 3532/3532 [00:00<00:00, 708376.69it/s]


vehicleName


...

100%|██████████████████████████████████████████████████████████████████████████| 3532/3532 [00:00<00:00, 885544.97it/s]


vehicleTransmission


100%|██████████████████████████████████████████████████████████████████████████| 3532/3532 [00:00<00:00, 708207.37it/s]


vehicleEngine


100%|██████████████████████████████████████████████████████████████████████████| 3532/3532 [00:00<00:00, 122023.65it/s]


In [600]:
df = df.drop(columns=[i for i in ['@context', '@type', 'offers'] if i in df.columns])

In [601]:
image_columns = [i for i in df.at[0, 'image'].keys()]
engine_columns = [i for i in df.at[0, 'vehicleEngine'].keys()]

In [602]:
for i in image_columns:
    if i in ['@context', '@type']:
        continue
    elif i not in df.columns:
        df[i] = df['image'].apply(lambda x: x[i].replace('\xa0', ' '))
    else:
        df['image_'+str(i)] = df['image'].apply(lambda x: x[i].replace('\xa0', ' '))
df = df.drop(columns=[i for i in ['@context', '@type', 'offers'] if i in df.columns])
df = df

In [603]:
for i in engine_columns:
    if i in ['@context', '@type']:
        continue
    elif i not in df.columns:
        df[i] = df['vehicleEngine'].apply(lambda x: x[i].replace('\xa0', ' '))
    else:
        df['engine_'+str(i)] = df['vehicleEngine'].apply(lambda x: x[i].replace('\xa0', ' '))
df = df.drop(columns=[i for i in ['@context', '@type', 'offers'] if i in df.columns])

In [604]:
df=df.drop(columns=['image', 'vehicleEngine'])

In [605]:
for i in df.columns:
    print(i)
    df[i] = df[i].progress_apply(lambda x: x.replace('"', '') if (isinstance(x, int) == False and x[0] == '"') else x)

vehicleName


100%|██████████████████████████████████████████████████████████████████████████| 3532/3532 [00:00<00:00, 590469.20it/s]


bodyType


100%|██████████████████████████████████████████████████████████████████████████| 3532/3532 [00:00<00:00, 590633.99it/s]


brand


...

100%|██████████████████████████████████████████████████████████████████████████| 3532/3532 [00:00<00:00, 877311.48it/s]


enginePower


100%|██████████████████████████████████████████████████████████████████████████| 3532/3532 [00:00<00:00, 590398.60it/s]


engine_fuelType


100%|██████████████████████████████████████████████████████████████████████████| 3532/3532 [00:00<00:00, 704134.31it/s]


In [606]:
df.head()

,vehicleName,bodyType,brand,color,fuelType,modelDate,description,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,author,contentUrl,datePublished,image_description,name,engine_name,engineDisplacement,enginePower,engine_fuelType
0,Lamborghini Urus I,внедорожник 5 дв.,LAMBORGHINI,серый,бензин,2017,Продам Ламборгини Урус 2021 года выпуска .\nК...,5,2021,ALLROAD_5_DOORS AUTOMATIC 4.0,автоматическая,Частное лицо,avatars.mds.yandex.net/get-autoru-vos/1597981/...,2021-11-10,Lamborghini Urus I серого цвета по цене 35 500...,Lamborghini Urus I серого цвета,4.0 AT,4.0 LTR,650 N12,бензи
1,BMW X5 30d IV (G05),внедорожник 5 дв.,BMW,белый,дизель,2018,Автомобиль в наличии! Последний на пневмоподве...,5,2021,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,Дикси BMW Барнаул,autoru.naydex.net/17gJBV451/d3896378Ae/c9T7onO...,2021-11-12,BMW X5 30d IV (G05) белого цвета по цене 9 305...,BMW X5 30d IV (G05) белого цвета,3.0 AT,3.0 LTR,249 N12,дизел
2,Kia Ceed III,универсал 5 дв.,KIA,чёрный,бензин,2018,- Автомобиль в отличном состоянии.\n- За все...,5,2019,WAGON_5_DOORS ROBOT 1.4,роботизированная,СИЛЬВЕР SELECT,autoru.naydex.net/17gJBV451/d3896378Ae/c9T7onO...,2021-11-13,Kia Ceed III чёрного цвета по цене 1 427 100 р...,Kia Ceed III чёрного цвета,1.4 AMT,1.4 LTR,140 N12,бензи
3,Opel Astra J Рестайлинг,седан,OPEL,серый,бензин,2012,чистый ухоженный автомобиль Комплект зимних ши...,4,2013,SEDAN AUTOMATIC 1.4,автоматическая,KK6969,autoru.naydex.net/17gJBV451/d3896378Ae/c9T7onO...,2021-10-11,Opel Astra J Рестайлинг серого цвета по цене 7...,Opel Astra J Рестайлинг серого цвета,1.4 AT,1.4 LTR,140 N12,бензи
4,Hyundai Creta II,внедорожник 5 дв.,HYUNDAI,белый,бензин,2021,Год выпуска: 2021. Пробег: новый. Кузов: внедо...,5,2021,ALLROAD_5_DOORS MECHANICAL 1.6,механическая,GN Service Hyundai Москва,autoru.naydex.net/17gJBV451/d3896378Ae/c9T7onO...,2021-09-10,Hyundai Creta II белого цвета по цене 1 270 00...,Hyundai Creta II белого цвета,1.6 MT,1.6 LTR,123 N12,бензи


In [607]:
### вычленяем цену из описания
df['price'] = 0
for i in range(len(df)):
    if ' рубл' in df.at[i, 'image_description']:
        df.at[i, 'price'] = df.at[i, 'image_description'].replace(df.at[i, 'name']+' по цене ', '')[:df.at[i, 'image_description'].replace(df.at[i, 'name']+' по цене ', '').find(' рубл')].replace(' ', '')
    else:
        continue

In [608]:
### убираем строки, где цена не нашлась - это порядка 10% выборки
df = df[df['price']>0].reset_index()
df = df.drop(columns=['index'])

#### Анализ признаков

In [609]:
# признаки из тестового датасета
required_feats = ['bodyType', 'brand', 'car_url', 'color', 'complectation_dict', 'description', 'engineDisplacement', 'enginePower',
'equipment_dict', 'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name', 'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency',
'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration', 'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль', 'Состояние', 'Таможня']



##### Убрать признаки (по анализу на kaggle)
<br>
- car_url - просто ссылка на страницу объявления, не несёт полезной для обучения информации<br>
- complectation_dict - null в 81% случаев в тестовом сете<br>
- description - текстовое описание, которое требует доп. обработки<br>
- equipment_dict - null в 29% случаев в тестовом сете и требует доп.обработки<br>
- image - просто ссылка<br>
- model_info - не интересно с точки зрения предсказания цены<br>
- parsing_unixtime - время парсинга - не интересно с точки зрения предсказания цены всех автомобилей<br>
- priceCurrency - одно уникальное значение по всей выборке<br>
- sell_id - не несёт полезной для обучения информации<br>
- super_gen - свод информации из уже имеющихся признаков<br>
- vendor - понятно из бренда<br>
- Владение - null в 65% случаев<br>
- ПТС - 87% одинаковых значений<br>
- Привод - уже содержится в vehicleTransmission<br>
- Руль - 96% одинаковых значений<br>
- Состояние - одно уникальное значение по всей выборке<br>
- Таможня - одно уникальное значение по всей выборке<br>

In [610]:
for i in ['car_url', 'complectation_dict', 'description', 'equipment_dict',
          'image', 'model_info', 'parsing_unixtime', 'vehicleConfiguration', 'priceCurrency',  'sell_id', 'super_gen', 'vendor', 'Владение', 'ПТС', 'Привод', 'Руль', 'Состояние', 'Таможня']:
    required_feats.remove(i)

In [611]:
required_feats

['bodyType',
 'brand',
 'color',
 'engineDisplacement',
 'enginePower',
 'fuelType',
 'mileage',
 'modelDate',
 'model_name',
 'name',
 'numberOfDoors',
 'productionDate',
 'vehicleTransmission',
 'Владельцы']

In [612]:
### проверям названия столбцов в учебном датасете
df.columns

Index(['vehicleName', 'bodyType', 'brand', 'color', 'fuelType', 'modelDate',
       'description', 'numberOfDoors', 'productionDate',
       'vehicleConfiguration', 'vehicleTransmission', 'author', 'contentUrl',
       'datePublished', 'image_description', 'name', 'engine_name',
       'engineDisplacement', 'enginePower', 'engine_fuelType', 'price'],
      dtype='object')

In [613]:
df = df.drop(columns = ['contentUrl'])

In [614]:
df.head()

,vehicleName,bodyType,brand,color,fuelType,modelDate,description,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,author,datePublished,image_description,name,engine_name,engineDisplacement,enginePower,engine_fuelType,price
0,Lamborghini Urus I,внедорожник 5 дв.,LAMBORGHINI,серый,бензин,2017,Продам Ламборгини Урус 2021 года выпуска .\nК...,5,2021,ALLROAD_5_DOORS AUTOMATIC 4.0,автоматическая,Частное лицо,2021-11-10,Lamborghini Urus I серого цвета по цене 35 500...,Lamborghini Urus I серого цвета,4.0 AT,4.0 LTR,650 N12,бензи,35500444
1,BMW X5 30d IV (G05),внедорожник 5 дв.,BMW,белый,дизель,2018,Автомобиль в наличии! Последний на пневмоподве...,5,2021,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,Дикси BMW Барнаул,2021-11-12,BMW X5 30d IV (G05) белого цвета по цене 9 305...,BMW X5 30d IV (G05) белого цвета,3.0 AT,3.0 LTR,249 N12,дизел,9305000
2,Kia Ceed III,универсал 5 дв.,KIA,чёрный,бензин,2018,- Автомобиль в отличном состоянии.\n- За все...,5,2019,WAGON_5_DOORS ROBOT 1.4,роботизированная,СИЛЬВЕР SELECT,2021-11-13,Kia Ceed III чёрного цвета по цене 1 427 100 р...,Kia Ceed III чёрного цвета,1.4 AMT,1.4 LTR,140 N12,бензи,1427100
3,Opel Astra J Рестайлинг,седан,OPEL,серый,бензин,2012,чистый ухоженный автомобиль Комплект зимних ши...,4,2013,SEDAN AUTOMATIC 1.4,автоматическая,KK6969,2021-10-11,Opel Astra J Рестайлинг серого цвета по цене 7...,Opel Astra J Рестайлинг серого цвета,1.4 AT,1.4 LTR,140 N12,бензи,777777
4,Hyundai Creta II,внедорожник 5 дв.,HYUNDAI,белый,бензин,2021,Год выпуска: 2021. Пробег: новый. Кузов: внедо...,5,2021,ALLROAD_5_DOORS MECHANICAL 1.6,механическая,GN Service Hyundai Москва,2021-09-10,Hyundai Creta II белого цвета по цене 1 270 00...,Hyundai Creta II белого цвета,1.6 MT,1.6 LTR,123 N12,бензи,1270000


In [615]:
df['monthPublished']=df.datePublished.astype('datetime64').dt.date.apply(lambda x: x.month)

In [616]:
df['monthPublished'].value_counts()

11    3021
10     241
9       86
8       15
7        6
3        2
6        2
5        1
Name: monthPublished, dtype: int64

In [617]:
df['engineDisplacement'] = df['engineDisplacement'].apply(lambda x: x.replace(' LTR', ''))
df['enginePower'] = df['enginePower'].apply(lambda x: x.replace(' N12', ''))

In [618]:
df['engineDisplacement'] = df['engineDisplacement'].apply(lambda x: 0 if x == '' else x)

In [619]:
df['engineDisplacement'] = df['engineDisplacement'].astype('float64')
df['enginePower'] = df['enginePower'].apply(lambda x: x.replace(' ', '')).astype('float64')

In [620]:
df['datePublished'] = df['datePublished'].astype('datetime64').dt.date

In [621]:
df['yearPublished'] = df['datePublished'].apply(lambda x: x.year)
df['dateFromProduction'] = df['yearPublished'] - df['productionDate']
# required_feats.append('dateFromProduction')

##### первая колонка из тестового сета, вторая - из обучающего

- bodyType = bodyType<br>
- brand = brand<br>
- color = color<br>
- engineDisplacement = engineDisplacement<br>
- enginePower = enginePower<br>
- fuelType = fuelType<br>
- mileage - отсутствует, уберём<br>
- modelDate = modelDate<br>
- model_name - нет, но можно сделать (из vehicleName уберём brand)<br>
- name = ближайшее - engine_name, но из тестовой выборки нужно убрать обозначение лошадиных сил и DSG<br>
- numberOfDoors = numberOfDoors<br>
- productionDate = productionDate<br>
- Владельцы - отсутствует, уберём<br>

In [622]:
### убираем признаки, которые не спарсились
required_feats.remove('mileage')
required_feats.remove('Владельцы')
# required_feats.remove('engineName')

In [623]:
a = 'LAMBO'
print(a.capitalize())

Lambo


In [624]:
df.head()

,vehicleName,bodyType,brand,color,fuelType,modelDate,description,numberOfDoors,productionDate,vehicleConfiguration,...,image_description,name,engine_name,engineDisplacement,enginePower,engine_fuelType,price,monthPublished,yearPublished,dateFromProduction
0,Lamborghini Urus I,внедорожник 5 дв.,LAMBORGHINI,серый,бензин,2017,Продам Ламборгини Урус 2021 года выпуска .\nК...,5,2021,ALLROAD_5_DOORS AUTOMATIC 4.0,...,Lamborghini Urus I серого цвета по цене 35 500...,Lamborghini Urus I серого цвета,4.0 AT,4.0,650.0,бензи,35500444,11,2021,0
1,BMW X5 30d IV (G05),внедорожник 5 дв.,BMW,белый,дизель,2018,Автомобиль в наличии! Последний на пневмоподве...,5,2021,ALLROAD_5_DOORS AUTOMATIC 3.0,...,BMW X5 30d IV (G05) белого цвета по цене 9 305...,BMW X5 30d IV (G05) белого цвета,3.0 AT,3.0,249.0,дизел,9305000,11,2021,0
2,Kia Ceed III,универсал 5 дв.,KIA,чёрный,бензин,2018,- Автомобиль в отличном состоянии.\n- За все...,5,2019,WAGON_5_DOORS ROBOT 1.4,...,Kia Ceed III чёрного цвета по цене 1 427 100 р...,Kia Ceed III чёрного цвета,1.4 AMT,1.4,140.0,бензи,1427100,11,2021,2
3,Opel Astra J Рестайлинг,седан,OPEL,серый,бензин,2012,чистый ухоженный автомобиль Комплект зимних ши...,4,2013,SEDAN AUTOMATIC 1.4,...,Opel Astra J Рестайлинг серого цвета по цене 7...,Opel Astra J Рестайлинг серого цвета,1.4 AT,1.4,140.0,бензи,777777,10,2021,8
4,Hyundai Creta II,внедорожник 5 дв.,HYUNDAI,белый,бензин,2021,Год выпуска: 2021. Пробег: новый. Кузов: внедо...,5,2021,ALLROAD_5_DOORS MECHANICAL 1.6,...,Hyundai Creta II белого цвета по цене 1 270 00...,Hyundai Creta II белого цвета,1.6 MT,1.6,123.0,бензи,1270000,9,2021,0


In [625]:
### добавляем признаки, которые несложно создать
df['model_name'] = str()
for i in range(len(df)):
    if df.at[i, 'brand'] in df.at[i, 'model_name']:
        df.at[i, 'model_name'] = df.at[i, 'vehicleName'].replace(df.at[i, 'brand'] + ' ', "")
    else:
        df.at[i, 'model_name'] = df.at[i, 'vehicleName'].replace(df.at[i, 'brand'].capitalize() + ' ', "")

In [626]:
df = df.drop(columns=['name'])
df = df.rename(columns={'engine_name':'name'})

In [627]:
test_df = pd.read_csv('test.csv')

In [628]:
def name_clear(text):
    text = text.replace('DSG ', '')
    text = text[ :text.find('(')-1]
    return text
    
test_df['name'] = test_df['name'].apply(name_clear)

In [629]:
test_df.head()

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,...,LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,...,LIFTBACK ROBOT 1.8,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
3,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,коричневый,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6 LTR,110 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,...,LIFTBACK AUTOMATIC 1.6,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
4,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,NaN,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8 LTR,152 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK AUTOMATIC 1.8,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


In [630]:
test_df['engineDisplacement'] = test_df['engineDisplacement'].apply(lambda x: x.replace(' LTR', ''))
test_df['enginePower'] = test_df['enginePower'].apply(lambda x: x.replace(' N12', ''))
test_df['engineDisplacement'] = test_df['engineDisplacement'].apply(lambda x: 0 if x == '' else x)
test_df['engineDisplacement'] = test_df['engineDisplacement'].astype('float32')
test_df['enginePower'] = test_df['enginePower'].apply(lambda x: x.replace(' ', '')).astype('float64')

In [631]:
test_df.head()

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2,105.0,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,...,LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6,110.0,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8,152.0,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,...,LIFTBACK ROBOT 1.8,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
3,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,коричневый,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6,110.0,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,...,LIFTBACK AUTOMATIC 1.6,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
4,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,NaN,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8,152.0,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK AUTOMATIC 1.8,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


In [632]:
y = df['price']

In [633]:
### Нужно скорректировать целевую переменную на дату парсинга, если прошло много времени
test_df['parsing_dt'] = test_df['parsing_unixtime'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
test_df['parsing_dt'] = test_df['parsing_dt'].astype('datetime64').dt.date
test_df['parsing_dt'].value_counts() ### всё было спарсено в октябре 2020 года

2020-10-21    15907
2020-10-20     6813
2020-10-19     6441
2020-10-25     2944
2020-10-24     2123
2020-10-26      458
Name: parsing_dt, dtype: int64

In [634]:
test_df['yearPublished'] = test_df['parsing_dt'].apply(lambda x: x.year)
test_df['dateFromProduction'] = test_df['yearPublished'] - test_df['productionDate']

###### Источник: "Автомобили с пробегом в октябре подорожали на 4%" (https://www.autostat.ru/news/49774/) © Автостат.
        
При этом по сравнению с показателем годичной давности (октябрь 2020-го) цены на вторичном рынке в среднем увеличились на 44%. То есть, в деньгах подержанные автомобили за год стали дороже на 267 тыс. рублей.

In [635]:
y = y / (1+0.44) ### приводим цены к нужному виду

In [636]:
test_df = test_df[[i for i in required_feats if i in test_df.columns]]
df = df[[i for i in required_feats if i in df.columns]]

In [637]:
test_df=test_df.drop(columns=['name'])
df = df.drop(columns=['name'])

In [638]:
df.shape, test_df.shape

((3374, 11), (34686, 11))

In [639]:
df.head()

,bodyType,brand,color,engineDisplacement,enginePower,fuelType,modelDate,model_name,numberOfDoors,productionDate,vehicleTransmission
0,внедорожник 5 дв.,LAMBORGHINI,серый,4.0,650.0,бензин,2017,Urus I,5,2021,автоматическая
1,внедорожник 5 дв.,BMW,белый,3.0,249.0,дизель,2018,BMW X5 30d IV (G05),5,2021,автоматическая
2,универсал 5 дв.,KIA,чёрный,1.4,140.0,бензин,2018,Ceed III,5,2019,роботизированная
3,седан,OPEL,серый,1.4,140.0,бензин,2012,Astra J Рестайлинг,4,2013,автоматическая
4,внедорожник 5 дв.,HYUNDAI,белый,1.6,123.0,бензин,2021,Creta II,5,2021,механическая


In [640]:
df = pd.get_dummies(df, columns=[i for i in df.columns if i not in ['modelDate', 'numberOfDoors', 'productionDate', 'engineDisplacement', 'enginePower', 'name', 'dateFromProduction']])
test_df = pd.get_dummies(test_df, columns=[i for i in test_df.columns if i not in ['modelDate', 'numberOfDoors', 'productionDate', 'engineDisplacement', 'enginePower', 'name', 'dateFromProduction']])

In [641]:
df.shape, test_df.shape

((3374, 1036), (34686, 610))

In [642]:
### проверяем, что количество столбцов в обоих датасетах совпадает
for i in test_df.columns:
    if i not in df.columns:
        df[i] = 0

for i in df.columns:
    if i not in test_df.columns:
        test_df[i] = 0

In [643]:
df.shape, test_df.shape

((3374, 1584), (34686, 1584))

In [644]:
### после анализа через метод наименьших квадратов из пакета Statsmodels
### убираем признаки, которые не отличаются от нуля на 5%-ном уровне значимости
### то есть, признаки с p-value больше 0,05
linreg_OLS = sm.OLS(y, sm.add_constant(df))
linreg_OLS = linreg_OLS.fit()
feats_to_keep = pd.DataFrame(linreg_OLS.pvalues).reset_index().dropna()
feats_to_keep = feats_to_keep[feats_to_keep[0] <= 0.05]

In [645]:
df = df[[i for i in feats_to_keep['index'].values if i != 'const']]
test_df = test_df[[i for i in feats_to_keep['index'].values if i != 'const']]

In [646]:
from sklearn.model_selection import GridSearchCV

In [647]:
param_grid = {'depth':[5, 10, 15, 20, 25, 30],
          'learning_rate':[0.3, 0.5, 0.7, 1]}

In [648]:
model = CatBoostRegressor(iterations=1000)
rgr = GridSearchCV(model, param_grid = param_grid, cv = 5)

In [ ]:
y_pred = rgr.fit(df, y).predict(test_df)

In [650]:
# forest = RandomForestRegressor(random_state=42)
# param_grid = {'min_samples_split':[2,3,4,5,10, 15], 'min_samples_leaf':[1, 2,3,5,10,15], 'max_features':['auto', 'sqrt', 'log2'],
#              'bootstrap':[True, False]}

# rgr = GridSearchCV(forest, param_grid = param_grid, cv=5, verbose=1)

# rgr = rgr.fit(df, y)
# y_pred = rgr.predict(test_df)

In [651]:
submission = pd.read_csv('test.csv')
submission = submission[['sell_id']]
submission['price'] = y_pred
submission.to_csv('Car_Price_prediction_submission.csv', index=False)